In [1]:
import os
# 设置环境变量，只让程序看到 GPU 2
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


import torch
import torch.nn as nn
import wandb
import random
import argparse
import numpy as np
from tqdm import tqdm
from transformers import BertModel, AutoModel
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据预处理函数

In [ ]:
from torch.utils.data import Dataset
import json

class BAE2025Dataset(Dataset):
    def __init__(
            self,
            data_path,
            labels={
                "Yes": 0,
                "To some extent": 1, 
                "No": 2,
            }
    ):
        self.data_path = data_path
        self.labels = labels
        self._get_data()
    
    def _get_data(self):
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        self.data = []
        for item in data:
            tutor_responses = item['tutor_responses']
            for response in tutor_responses.values():
                sent1 = item['conversation_history']
                sent2 = response['response']
                label = response['annotation']["Providing_Guidance"]
                if label in self.labels:
                    self.data.append(((sent1, sent2), self.labels[label]))
    
    def __len__(self):
        return len(self.data)
    
    def get_labels(self):
        return self.labels

    def __getitem__(self, idx):
        return self.data[idx]

# 数据加载函数

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

class BAE2025DataLoader:
    def __init__(
        self,
        dataset,
        batch_size=16,
        max_length=512,
        shuffle=True,
        drop_last=True,
        device=None,
        # tokenizer_name='chinese-bert-wwm-ext'
        # tokenizer_name='chinese-roberta-wwm-ext'
        # tokenizer_name='chinese-roberta-wwm-ext-large'
        # tokenizer_name='/mnt/cfs/huangzhiwei/pykt-moekt/SBM/bge-large-en-v1.5'
        # tokenizer_name='/mnt/cfs/huangzhiwei/BAE2025/models/bge-base-en-v1.5'
        tokenizer_name='/mnt/cfs/huangzhiwei/BAE2025/models/bert-base-uncased'
        # tokenizer_name='/mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base'
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.dataset = dataset
        self.batch_size = batch_size
        self.max_length = max_length
        self.shuffle = shuffle
        self.drop_last = drop_last

        if device is None:
            self.device = torch.device(
                'cuda' if torch.cuda.is_available() else 'cpu'
            )
        else:
            self.device = device

        self.loader = DataLoader(
            dataset=self.dataset,
            batch_size=self.batch_size,
            collate_fn=self.collate_fn,
            shuffle=self.shuffle,
            drop_last=self.drop_last
        )

    def collate_fn(self, data):
        sents = [i[0] for i in data]
        labels = [i[1] for i in data]

        # 修改这里，处理两个句子的情况
        data = self.tokenizer.batch_encode_plus(
            batch_text_or_text_pairs=[(sent[0], sent[1]) for sent in sents],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            return_length=True
        )
        input_ids = data['input_ids'].to(self.device)
        attention_mask = data['attention_mask'].to(self.device)
        token_type_ids = data['token_type_ids'].to(self.device)
        labels = torch.LongTensor(labels).to(self.device)

        return input_ids, attention_mask, token_type_ids, labels
        # return input_ids, attention_mask, labels


    def __iter__(self):
        for data in self.loader:
            yield data

    def __len__(self):
        return len(self.loader)



# check 一下数据是不是对的

In [4]:
# 校验上述数据加载器出来的数据是否正确，输出数据出来看一看
train_data_path = '../data/train.json'
val_data_path = '../data/valid.json'
train_dataset = BAE2025Dataset(train_data_path)
train_dataloader = BAE2025DataLoader(train_dataset, batch_size=1)
val_dataset = BAE2025Dataset(val_data_path)
val_dataloader = BAE2025DataLoader(val_dataset, batch_size=1)

cnt_train=0
for batch in train_dataloader:
    cnt_train += 1
    input_ids, attention_mask, token_type_ids, labels = batch
    # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, coarse_labels.shape, fine_labels.shape)
    # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    # break
print("train data size:", cnt_train)

cnt_val=0
for batch in val_dataloader:
    cnt_val += 1
    input_ids, attention_mask, token_type_ids, labels = batch
    # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, coarse_labels.shape, fine_labels.shape)
    # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    # break
print("val data size:", cnt_val)

train data size: 1975
val data size: 501


In [5]:
# 检查一下dataset中处理出来的数据是否正确
for data in train_dataset:
    sent, label = data
    # print(sent, coarse_label, fine_label, sent_id)
    print(sent, label)

('Tutor: Hi, could you please provide a step-by-step solution for the question below? The question is: Tyson decided to make muffaletta sandwiches for the big game.\xa0\xa0Each sandwich required 1 pound each of meat and cheese and would serve 4 people.\xa0\xa0There would be 20 people in total watching the game.\xa0\xa0The meat cost $7.00 per pound and the cheese cost $3.00 per pound.\xa0\xa0How much money would he spend on the meat and cheese to make enough sandwiches to serve 20 people?\xa0\n\xa0Student: To serve 20 people, Tyson needs to make 20/4 = 5 sandwiches.\nEach sandwich requires 1+1 = 2 pounds of meat and cheese.\nFor 5 sandwiches, he needs a total of 2 x 5 = 10 pounds of meat and cheese.\nThe cost of 10 pounds of meat is 10 x $7.00 = $70.\nThe cost of 10 pounds of cheese is 10 x $3.00 = $30.\nThe total cost of meat and cheese is $70 + $30 = $100.\n\xa0100\xa0\n\xa0Tutor: How many pounds of meat are needed for each sandwich?\xa0\n\xa0Student: Each sandwich requires 1 pound of

# 模型代码

In [6]:
# import torch
# import torch.nn as nn
# from transformers import BertModel

# class BertClassifier(nn.Module):
#     def __init__(self, pretrained_model_name, num_classes=3, freeze_pooler=False, dropout=0.2):
#         super().__init__()

#         self.freeze_pooler = freeze_pooler
#         self.bert = BertModel.from_pretrained(pretrained_model_name)
#         self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
#         if freeze_pooler:
#             for param in self.bert.pooler.parameters():
#                 param.requires_grad = False

#         # self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
#         self.classifier = nn.Sequential(
#             nn.Linear(self.bert.config.hidden_size, 256),
#             nn.ReLU(),
#             nn.Linear(256, num_classes)
#         )

#     def forward(self, input_ids, attention_mask, token_type_ids):
#     # def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         # outputs = self.bert(input_ids, attention_mask=attention_mask)

#         pooled_output = outputs.pooler_output

#         pooled_output = self.dropout(pooled_output)

#         logits = self.classifier(pooled_output)
        
#         # logits = torch.sigmoid(logits)

#         return logits





import torch
import torch.nn as nn
from transformers import BertModel


class BertClassificationHead(nn.Module):
    def __init__(self, hidden_size=1024, num_classes=3, dropout_prob=0.3):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.out_proj = nn.Linear(hidden_size, num_classes)  # (输入维度，输出维度)

    def forward(self, features, **kwargs):
        x = features[-1][:, 0, :]  # features[-1]是一个三维张量，其维度为[批次大小, 序列长度, 隐藏大小]。
        x = self.dropout(x)  # 这是一种正则化技术，用于防止模型过拟合。在训练过程中，它通过随机将输入张量中的一部分元素设置为0，来增加模型的泛化能力。
        x = self.dense(x)  # 这是一个全连接层，它将输入特征映射到一个新的特征空间。这是通过学习一个权重矩阵和一个偏置向量，并使用它们对输入特征进行线性变换来实现的，方便后续可以引入非线性变换。
        x = torch.tanh(x)  # 这是一个激活函数，它将线性层的输出转换为非线性，使得模型可以学习并表示更复杂的模式。
        x = self.dropout(x)  # 增加模型的泛化能力。
        x = self.out_proj(x)  # 这是最后的全连接层，它将特征映射到最终的输出空间。在这个例子中，输出空间的维度等于分类任务的类别数量。
        return x
    

class BertClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_classes=3, freeze_pooler=0, dropout=0.3, hidden_size=768):
        super().__init__()
        
        self.bert = BertModel.from_pretrained(pretrained_model_name, output_hidden_states=True)
        
        # 冻结BERT底层，保留顶层微调
        if freeze_pooler > 0:
            modules = [self.bert.embeddings, *self.bert.encoder.layer[:freeze_pooler]]
            for module in modules:
                for param in module.parameters():
                    param.requires_grad = False
        
        self.dropout = nn.Dropout(dropout)
        
        # 获取bert隐藏层大小
        bert_hidden_size = self.bert.config.hidden_size
        
        self.classifier = BertClassificationHead(
            hidden_size=self.bert.config.hidden_size,
            num_classes=3,  # 三分类任务
            dropout_prob=dropout
        )
        
    def forward(self, input_ids, attention_mask, token_type_ids=None):
        # 获取BERT输出
        outputs = self.bert(
            input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
            # output_hidden_states=True  # 获取所有隐藏层
        )
        
        # 使用[CLS]表示的序列表示
        # pooled_output = outputs.pooler_output
        
        # 可选：结合最后四层的[CLS]表示以获取更丰富的信息
        # last_4_layers = outputs.hidden_states[-4:]
        # cls_embeddings = torch.stack([layer[:, 0, :] for layer in last_4_layers], dim=0)
        # pooled_output = torch.mean(cls_embeddings, dim=0)  # 平均最后四层
        
        # 应用dropout
        # pooled_output = self.dropout1(pooled_output)
        
        # 分类
        logits = self.classifier(outputs.hidden_states)
        
        return logits

# 训练参数设置

In [7]:
import os
import wandb
import random
import argparse
from tqdm import tqdm

import torch
import torch.nn as nn
import numpy as np
from transformers import AdamW
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# 如果在Jupyter Notebook中运行，可以使用这个自定义参数函数替代argparser
def get_default_configs():
    """在Jupyter环境中使用的默认配置，避免argparse解析错误"""
    class Args:
        def __init__(self):
            # self.model_name = '/mnt/cfs/huangzhiwei/pykt-moekt/SBM/bge-large-en-v1.5'
            # self.model_name = "/mnt/cfs/huangzhiwei/BAE2025/models/ModernBERT-large"
            # self.model_name = '/mnt/cfs/huangzhiwei/pykt-moekt/SBM/xlm-roberta-large'
            # self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/bge-base-en-v1.5'
            self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/bert-base-uncased'
            # self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base'
            self.num_classes = 3
            self.dropout = 0.3
            self.freeze_pooler = 8
            self.batch_size = 16
            self.max_length = 512
            self.lr = 1e-5
            self.epochs = 50
            self.device = device
            self.name = None
            self.seed = 42
            self.data_path = '../data/train.json'
            self.val_data_path = '../data/valid.json'
            self.checkpoint_dir = 'checkpoints_3'
            self.patience = 8
            self.exp_name = 'BAE2025_track4_bert'
    return Args()


# 训练函数

In [8]:
def train(configs):
    
    # 设置随机种子
    random.seed(configs.seed)
    np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 创建检查点目录
    checkpoint_dir = os.path.join(configs.checkpoint_dir, configs.exp_name)
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # 为保存混淆矩阵创建目录 - 分别为训练集和验证集创建
    train_plot_dir = os.path.join(checkpoint_dir, 'plots', 'train')
    val_plot_dir = os.path.join(checkpoint_dir, 'plots', 'val')
    os.makedirs(train_plot_dir, exist_ok=True)
    os.makedirs(val_plot_dir, exist_ok=True)
    
    # 创建用于保存预测结果的JSON目录
    predictions_dir = os.path.join(checkpoint_dir, 'predictions')
    os.makedirs(predictions_dir, exist_ok=True)
    
    # 加载数据集
    train_dataset = BAE2025Dataset(configs.data_path)
    val_dataset = BAE2025Dataset(configs.val_data_path)    

    # 创建数据加载器
    train_dataloader = BAE2025DataLoader(
        dataset=train_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=True,
        drop_last=True,
        device=configs.device,
        tokenizer_name=configs.model_name
    )

    val_dataloader = BAE2025DataLoader(
        dataset=val_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=False,
        drop_last=False,
        device=configs.device,
        tokenizer_name=configs.model_name
    )
    
    # 创建模型
    model = BertClassifier(
        pretrained_model_name=configs.model_name,
        num_classes=configs.num_classes,
        freeze_pooler=configs.freeze_pooler,
        dropout=configs.dropout
    ).to(configs.device)

    criterion = nn.CrossEntropyLoss()

    # 定义优化器
    optimizer = AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=configs.lr
    )

    # 初始化最佳验证损失和早停计数器
    best_val_acc = 0.0
    best_val_f1 = 0.0  # 添加F1分数作为评估指标
    best_val_loss = float('inf')
    patience_counter = 0
    
    # 定义类别名称
    class_names = ['Yes', 'To some extent', 'No']
    
    # 添加F1计算所需的库
    from sklearn.metrics import f1_score, confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import json
    
    # 训练循环
    for epoch in range(configs.epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        train_preds = []
        train_labels_list = []
        train_probs = []  # 存储预测概率
        
        with tqdm(
            train_dataloader,
            total=len(train_dataloader),
            desc=f'Epoch {epoch + 1}/{configs.epochs}',
            unit='batch',
            ncols=100
        ) as pbar:
            for input_ids, attention_mask, token_type_ids, labels in pbar:
                optimizer.zero_grad()
                
                # 前向传播
                logits = model(input_ids, attention_mask, token_type_ids)
                
                # 计算损失 - 确保labels是长整型
                labels = labels.long()
                loss = criterion(logits, labels)
                
                # 反向传播
                loss.backward()
                optimizer.step()
                
                # 计算预测概率
                probs = torch.softmax(logits, dim=1).detach().cpu().numpy().tolist()
                
                # 获取预测标签
                preds = logits.argmax(dim=1)
                accuracy = (preds == labels).float().mean()
                accuracy_all = (preds == labels).float().sum()
                
                # 收集预测结果、概率和真实标签
                train_preds.extend(preds.cpu().numpy().tolist())
                train_labels_list.extend(labels.cpu().numpy().tolist())
                train_probs.extend(probs)
                
                train_loss += loss.item()
                train_acc += accuracy_all.item()
                
                pbar.set_postfix(
                    loss=f'{loss.item():.3f}',
                    accuracy=f'{accuracy.item():.3f}'
                )
        
        train_loss = train_loss / len(train_dataloader)
        train_acc = train_acc / len(train_dataset)
        
        # 计算训练集的F1分数 - 使用macro平均以处理多分类
        train_f1 = f1_score(train_labels_list, train_preds, average='macro')
        
        print(f'Training Loss: {train_loss:.4f}')
        print(f'Training Accuracy: {train_acc:.4f}')
        print(f'Training F1 Score: {train_f1:.4f}')
        
        # 保存训练集的预测结果到JSON
        train_predictions = {
            "epoch": epoch + 1,
            "loss": train_loss,
            "accuracy": train_acc,
            "f1_score": train_f1,
            "predictions": [
                {
                    "true_label": int(label),
                    "predicted_label": int(pred),
                    "probabilities": {
                        class_names[0]: prob[0],
                        class_names[1]: prob[1],
                        class_names[2]: prob[2]
                    }
                }
                for label, pred, prob in zip(train_labels_list, train_preds, train_probs)
            ]
        }
        
        # 保存训练预测结果到JSON文件
        train_json_path = os.path.join(predictions_dir, f'train_predictions_epoch_{epoch+1}.json')
        with open(train_json_path, 'w', encoding='utf-8') as f:
            json.dump(train_predictions, f, indent=2, ensure_ascii=False)
        
        # 创建训练集的混淆矩阵
        # 创建三个二分类混淆矩阵（两两类别之间）
        class_pairs = [
            ([0, 1], ['Yes', 'To some extent']),  # Yes vs To some extent
            ([0, 2], ['Yes', 'No']),              # Yes vs No
            ([1, 2], ['To some extent', 'No'])    # To some extent vs No
        ]
        
        for classes_idx, classes_names in class_pairs:
            # 筛选出对应两个类别的预测和标签
            mask = np.isin(np.array(train_labels_list), classes_idx)
            filtered_preds = np.array(train_preds)[mask]
            filtered_labels = np.array(train_labels_list)[mask]
            
            # 创建混淆矩阵
            cm = confusion_matrix(filtered_labels, filtered_preds, labels=classes_idx)
            
            # 计算此对类别的准确率和F1分数
            pair_mask = np.isin(np.array(train_labels_list), classes_idx)
            pair_acc = np.mean(np.array(train_preds)[pair_mask] == np.array(train_labels_list)[pair_mask])
            # 计算二分类F1分数
            pair_f1 = f1_score(
                np.array(train_labels_list)[pair_mask], 
                np.array(train_preds)[pair_mask], 
                labels=classes_idx, 
                average='macro'
            )
            
            # 绘制混淆矩阵
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=[classes_names[i == classes_idx[1]] for i in classes_idx],
                        yticklabels=[classes_names[i == classes_idx[1]] for i in classes_idx])
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title(f'Train: {classes_names[0]} vs {classes_names[1]}\nAcc: {pair_acc:.4f}, F1: {pair_f1:.4f}')
            
            # 保存图表
            matrix_path = os.path.join(train_plot_dir, f'cm_{classes_names[0].replace(" ", "_")}_{classes_names[1].replace(" ", "_")}_epoch_{epoch+1}.png')
            plt.savefig(matrix_path)
            plt.close()
            
        # 创建完整的三分类混淆矩阵
        cm_full = confusion_matrix(train_labels_list, train_preds, labels=[0, 1, 2])
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm_full, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names,
                    yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Train: Full Confusion Matrix\nAcc: {train_acc:.4f}, F1: {train_f1:.4f}')
        
        # 保存完整混淆矩阵
        matrix_path = os.path.join(train_plot_dir, f'cm_full_epoch_{epoch+1}.png')
        plt.savefig(matrix_path)
        plt.close()
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        val_corrects = 0.0
        val_preds = []
        val_labels_list = []
        val_probs = []  # 存储验证集预测概率

        with torch.no_grad():
            for input_ids, attention_mask, token_type_ids, labels in val_dataloader:
                # 确保labels是长整型
                labels = labels.long()
                
                # 前向传播
                logits = model(input_ids, attention_mask, token_type_ids)
                
                # 计算损失
                loss = criterion(logits, labels)
                val_loss += loss.item()
                
                # 计算预测概率
                probs = torch.softmax(logits, dim=1).detach().cpu().numpy().tolist()
                
                # 获取预测标签
                preds = logits.argmax(dim=1)
                accuracy = (preds == labels).float().sum()
                val_corrects += accuracy
                
                # 收集预测结果、概率和真实标签
                val_preds.extend(preds.cpu().numpy().tolist())
                val_labels_list.extend(labels.cpu().numpy().tolist())
                val_probs.extend(probs)
        
        val_loss = val_loss / len(val_dataloader)
        val_acc = val_corrects.double() / len(val_dataset)
        
        # 计算验证集的F1分数
        val_f1 = f1_score(val_labels_list, val_preds, average='macro')
        
        print('Validation Loss: {:.4f} Acc: {:.4f} F1: {:.4f}'.format(val_loss, val_acc, val_f1))
        
        # 保存验证集的预测结果到JSON
        val_predictions = {
            "epoch": epoch + 1,
            "loss": val_loss,
            "accuracy": val_acc.item(),
            "f1_score": val_f1,
            "predictions": [
                {
                    "true_label": int(label),
                    "predicted_label": int(pred),
                    "probabilities": {
                        class_names[0]: prob[0],
                        class_names[1]: prob[1],
                        class_names[2]: prob[2]
                    }
                }
                for label, pred, prob in zip(val_labels_list, val_preds, val_probs)
            ]
        }
        
        # 保存验证预测结果到JSON文件
        val_json_path = os.path.join(predictions_dir, f'val_predictions_epoch_{epoch+1}.json')
        with open(val_json_path, 'w', encoding='utf-8') as f:
            json.dump(val_predictions, f, indent=2, ensure_ascii=False)
        
        # 创建验证集三个二分类混淆矩阵（两两类别之间）
        class_pairs = [
            ([0, 1], ['Yes', 'To some extent']),  # Yes vs To some extent
            ([0, 2], ['Yes', 'No']),              # Yes vs No
            ([1, 2], ['To some extent', 'No'])    # To some extent vs No
        ]
        
        for classes_idx, classes_names in class_pairs:
            # 筛选出对应两个类别的预测和标签
            mask = np.isin(np.array(val_labels_list), classes_idx)
            filtered_preds = np.array(val_preds)[mask]
            filtered_labels = np.array(val_labels_list)[mask]
            
            # 创建混淆矩阵
            cm = confusion_matrix(filtered_labels, filtered_preds, labels=classes_idx)
            
            # 计算此对类别的准确率和F1分数
            pair_mask = np.isin(np.array(val_labels_list), classes_idx)
            pair_acc = np.mean(np.array(val_preds)[pair_mask] == np.array(val_labels_list)[pair_mask])
            # 计算二分类F1分数
            pair_f1 = f1_score(
                np.array(val_labels_list)[pair_mask], 
                np.array(val_preds)[pair_mask], 
                labels=classes_idx, 
                average='macro'
            )
            
            # 绘制混淆矩阵
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=[classes_names[i == classes_idx[1]] for i in classes_idx],
                        yticklabels=[classes_names[i == classes_idx[1]] for i in classes_idx])
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title(f'Val: {classes_names[0]} vs {classes_names[1]}\nAcc: {pair_acc:.4f}, F1: {pair_f1:.4f}')
            
            # 保存图表
            matrix_path = os.path.join(val_plot_dir, f'cm_{classes_names[0].replace(" ", "_")}_{classes_names[1].replace(" ", "_")}_epoch_{epoch+1}.png')
            plt.savefig(matrix_path)
            plt.close()
            
        # 创建完整的三分类混淆矩阵
        cm_full = confusion_matrix(val_labels_list, val_preds, labels=[0, 1, 2])
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm_full, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names,
                    yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Val: Full Confusion Matrix\nAcc: {val_acc:.4f}, F1: {val_f1:.4f}')
        
        # 保存完整混淆矩阵
        matrix_path = os.path.join(val_plot_dir, f'cm_full_epoch_{epoch+1}.png')
        plt.savefig(matrix_path)
        plt.close()
        
        # 检查是否保存模型并判断是否需要早停
        # 使用F1分数作为主要指标
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_val_acc = val_acc
            
            # 保存模型
            state_dict = model.state_dict()
            torch.save(state_dict, os.path.join(checkpoint_dir, 'best_model_f1.pt'))
            print(f'New best model saved with F1: {best_val_f1:.4f}, Acc: {best_val_acc:.4f}')
            
            # 保存最佳模型的预测结果
            best_model_predictions = {
                "train": train_predictions,
                "val": val_predictions
            }
            best_predictions_path = os.path.join(predictions_dir, 'best_model_predictions.json')
            with open(best_predictions_path, 'w', encoding='utf-8') as f:
                json.dump(best_model_predictions, f, indent=2, ensure_ascii=False)
            
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= configs.patience:
                print(f'Early stopping triggered after {epoch+1} epochs.')
                break

        model.train()
        
# 在以下主函数中添加判断Jupyter环境的逻辑
if __name__ == '__main__':
    # 判断是否在Jupyter环境中运行
    try:
        # 检查是否在Jupyter中运行
        get_ipython = globals().get('get_ipython', None)
        if get_ipython and 'IPKernelApp' in get_ipython().config:
            # 在Jupyter环境中运行，使用默认配置
            print("Running in Jupyter environment, using default configs")
            configs = get_default_configs()
        else:
            # 在命令行环境中运行，使用argparse
            configs = argparser()
    except:
        # 任何异常都使用argparse处理
        configs = argparser()
    
    # 设置实验名称
    if configs.name is None:
        configs.exp_name = \
            f'{os.path.basename(configs.model_name)}' + \
            f'{"_fp" if configs.freeze_pooler else ""}' + \
            f'_b{configs.batch_size}_e{configs.epochs}' + \
            f'_len{configs.max_length}_lr{configs.lr}'
    else:
        configs.exp_name = configs.name
    
    # 设置设备
    if configs.device is None:
        configs.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu'
        )
    
    # 调用训练函数
    train(configs)

Running in Jupyter environment, using default configs


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/50: 100%|██████████████████| 123/123 [00:10<00:00, 11.25batch/s, accuracy=0.688, loss=0.930]


Training Loss: 0.9581
Training Accuracy: 0.5878
Training F1 Score: 0.3560
Validation Loss: 0.8338 Acc: 0.6687 F1: 0.4265
New best model saved with F1: 0.4265, Acc: 0.6687


Epoch 2/50: 100%|██████████████████| 123/123 [00:10<00:00, 11.71batch/s, accuracy=0.500, loss=1.000]


Training Loss: 0.8540
Training Accuracy: 0.6466
Training F1 Score: 0.4491
Validation Loss: 0.8012 Acc: 0.6946 F1: 0.4789
New best model saved with F1: 0.4789, Acc: 0.6946


...

Training Loss: 0.7444
Training Accuracy: 0.6765
Training F1 Score: 0.5353
Validation Loss: 0.8380 Acc: 0.6507 F1: 0.5308


Epoch 5/50: 100%|██████████████████| 123/123 [00:10<00:00, 11.62batch/s, accuracy=0.812, loss=0.510]


Training Loss: 0.6790
Training Accuracy: 0.7154
Training F1 Score: 0.6104
Validation Loss: 0.8523 Acc: 0.6806 F1: 0.5383


Epoch 6/50:   8%|█▌                 | 10/123 [00:00<00:10, 10.57batch/s, accuracy=1.000, loss=0.290]


KeyboardInterrupt: 